In [107]:
import os

import yaml
from config import SETTINGS
from pathlib import Path
import pandas as pd

import seabeepy as sb

# Refactor `config.seabee.yaml` for niva-tidy missions

In [76]:
# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)

In [77]:
# Parent directories containing flight folders to process
base_dirs = [
    Path(r"/home/kim/work/seabee/test-detection/niva-tidy/2022"),
    Path(r"/home/kim/work/seabee/test-detection/niva-tidy/2023"),
]


In [96]:
def template_config() -> dict:
    return dict(
    grouping=None,
    area=None,
    datetime=None,
    spectrum_type=None,
    elevation=None,
    project=None,
    nfiles=None,
    organisation=None,
    creator_name=None,
    theme="habitat",
    mosaic=False,
    publish=True,
    #classify=True,
    )


def niva_name_to_config(name: str):

    org, date, group, area, spect, elev = name.split("_")

    return dict(
        grouping = group,
        area = area,
        datetime = date,
        spectrum_type = spect.lower() if spect.lower() != "msi" else "ms",
        organisation = org,
        elevation = int(elev)
    )


def parse_config(dir_path: Path):
    conf = {}
    config_path = dir_path / "config.seabee.yaml"
    if config_path.exists():
        with open(config_path, "r") as f:
            conf = yaml.safe_load(f)
    return conf


def write_new_config(path: Path, data):
    with open(path, "w") as yaml_file:
        yaml.dump(data, yaml_file, default_flow_style=False, allow_unicode=True)


def merge_conf(folder_conf: dict, existing_conf: dict) -> dict:
    """Merge the folder_conf with the existing_conf

    It is assumed that the folder_conf is the correct one for most fields
    """

    new_config = template_config()
    for key, val in folder_conf.items():
        new_config[key] = val
    for key in ["creator_name", "project", "mosaic", "publish", "theme"]:
        if key in existing_conf:
            if key == "creator_name" and "_" in existing_conf[key]:
                new_config[key] = " ".join(existing_conf[key].split("_"))
            else:
                new_config[key] = existing_conf[key]
            
    return new_config

In [97]:
dir_list = [p for p in base_dirs for p in p.iterdir() if p.is_dir()]

In [102]:
tmp_dir = Path("./tmp")
tmp_dir.mkdir(exist_ok=True)

In [109]:

tmp_dir = Path(r"tmp")
mission_dict = dict(folder_name= [], layer_name = [])
mission_dict.update({k:[] for k in template_config().keys()})
for dir_path in dir_list:
    # Update config
    mission_name = os.path.split(dir_path)[-1]
    existing_conf = parse_config(dir_path)
    folder_conf = niva_name_to_config(mission_name)
    
    new_conf = merge_conf(folder_conf, existing_conf)
    new_conf["nfiles"] = len(sb.ortho.list_images(dir_path / "images", verbose=False))
    # Save locally and validate

    sb.ortho.CONFIG_SCHEMA.validate(new_conf)
    write_new_config(tmp_dir / "config.seabee.yaml", new_conf)
    layer_name = sb.ortho.get_layer_name(tmp_dir)
    mission_dict["folder_name"].append(mission_name)
    mission_dict["layer_name"].append(layer_name)


    tmp_file = tmp_dir / f"{mission_name}.yaml"
    write_new_config(tmp_file, new_conf)


TypeError: 'NoneType' object is not subscriptable

In [108]:
pd.DataFrame(mission_dict)

,folder_name,layer_name
0,niva_202208190944_kelpmap_vega-n_rgb_120,kelpmap_vega-n_202208190944_rgb_120m
1,ntnu_202208311435_runde_remoy_hsi_40,runde_remoy_202208311435_hsi_40m
2,spectrofly_202208180945_kelpmap_vega-s_msi_120,kelpmap_vega-s_202208180945_ms_120m
3,niva_202309271147_ascomap_bringnes_msi_30,ascomap_bringnes_202309271147_ms_30m
4,niva_202310031129_io_husviksbaen_msi_80,io_husviksbaen_202310031129_ms_80m
5,niva_202307140902_larvik_olberg-all_msi_100,larvik_olberg-all_202307140902_ms_100m
